### Converting new bookings forecasts for deferred revenue model

Since Karen Burgess changed roles, I now have two sources of data for the bookings forecast

DX - 
DME - 

The data files they provide are very different and I need to change them to get one bookings dataframe


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pickle


### DME bookings

In [2]:
filename = r'/Volumes/Treasury/Financial_Database/Deferred_Revenue/Inputs/DATA_2020_p12/DME_Bookings_FY21_Plan.xlsx'
sheetname = 'Raw'
df_DME = pd.read_excel(filename, sheetname)

In [3]:
df_DME.head(10)

,Metrics,Profit center,Market Area,Market Segement,GTM,Q1 2021,Q2 2021,Q3 2021,Q4 2021,2021
0,Total Subscription Attrition,EB10 - Creative,AMER (G),COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06
1,Total Subscription Attrition,EB10 - Creative,AMER (G),EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06
2,Total Subscription Attrition,EB10 - Creative,AMER (G),GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06
3,Total Subscription Attrition,EB10 - Creative,AMER (G),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04
4,Total Subscription Attrition,EB10 - Creative,North America (R),COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06
5,Total Subscription Attrition,EB10 - Creative,North America (R),EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06
6,Total Subscription Attrition,EB10 - Creative,North America (R),GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06
7,Total Subscription Attrition,EB10 - Creative,North America (R),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04
8,Total Subscription Attrition,EB10 - Creative,Canada (MA),COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05
9,Total Subscription Attrition,EB10 - Creative,Canada (MA),EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05


In [4]:
# Clean Column Names
df_DME.columns

Index(['Metrics', 'Profit center', 'Market Area', 'Market Segement', 'GTM',
       'Q1 2021', 'Q2 2021', 'Q3 2021', 'Q4 2021', '2021'],
      dtype='object')

In [5]:
df_DME = df_DME.rename(columns = {'Metrics': 'metrics',
                        'Profit center': 'profit_center',
                        'Market Area': 'market_area',
                        'Market Segement': 'segment',
                        'Q1 2021':'Q1_2021',
                        'Q2 2021':'Q2_2021',
                        'Q3 2021':'Q3_2021',
                        'Q4 2021':'Q4_2021' 
                        })

In [6]:
df_DME.columns

Index(['metrics', 'profit_center', 'market_area', 'segment', 'GTM', 'Q1_2021',
       'Q2_2021', 'Q3_2021', 'Q4_2021', '2021'],
      dtype='object')

In [7]:
df_DME['market_area'].value_counts()

United States (MA)           224
North America (R)            224
AMER (G)                     224
ASIA (G)                     180
ANZ (R)                      172
Aus and New Zealand (MA)     172
SEA (R)                      154
Southeast Asia (MA)          154
Korea (MA)                   132
Korea (R)                    132
Greater China (R)            130
EMEA (G)                     118
China (MA)                   116
Northern Europe (R)          110
Canada (MA)                  110
Germany (MA)                 109
Central Europe (R)           109
Benelux (MA)                 108
India (R)                    108
India (MA)                   108
France (MA)                  108
Southwest Europe (R)         108
United Kingdom (MA)          106
Switzerland (MA)             105
Italy (MA)                   104
Middle East (MA)             104
Iberica (MA)                 104
Nordic (MA)                  102
Hong Kong & Taiwan (MA)       98
SSA & Israel (MA)             88
Latin Amer

In [8]:
df_DME.metrics.value_counts(dropna=False)

Net ACV                         1797
ASV & Usage Based               1510
Total Subscription Attrition    1096
Name: metrics, dtype: int64

## It looks like there are three seperate cuts of the data that are a hierarchy

1. G is GEO
2. R is region
3. MA is market Area

Going to split these into seperate sections and do successive fill forwards to cover all of them



In [9]:
# identify the characters in a string
df_DME['in_parens'] =  df_DME['market_area'].apply(lambda st: st[st.find("(")+1:st.find(")")])


In [10]:
df_DME['in_parens'].value_counts()

MA    2452
R     1379
G      572
Name: in_parens, dtype: int64

In [11]:
# now delete the parenthesis and the items between the parenthesis in the 'Market Area' column
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   metrics        4403 non-null   object 
 1   profit_center  4403 non-null   object 
 2   market_area    4403 non-null   object 
 3   segment        4403 non-null   object 
 4   GTM            4403 non-null   object 
 5   Q1_2021        4403 non-null   float64
 6   Q2_2021        4403 non-null   float64
 7   Q3_2021        4403 non-null   float64
 8   Q4_2021        4403 non-null   float64
 9   2021           4403 non-null   float64
 10  in_parens      4403 non-null   object 
dtypes: float64(5), object(6)
memory usage: 378.5+ KB


In [12]:
df_DME.head(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens
0,Total Subscription Attrition,EB10 - Creative,AMER (G),COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G
1,Total Subscription Attrition,EB10 - Creative,AMER (G),EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G
2,Total Subscription Attrition,EB10 - Creative,AMER (G),GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G
3,Total Subscription Attrition,EB10 - Creative,AMER (G),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G
4,Total Subscription Attrition,EB10 - Creative,North America (R),COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R
5,Total Subscription Attrition,EB10 - Creative,North America (R),EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06,R
6,Total Subscription Attrition,EB10 - Creative,North America (R),GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06,R
7,Total Subscription Attrition,EB10 - Creative,North America (R),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,R
8,Total Subscription Attrition,EB10 - Creative,Canada (MA),COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,MA
9,Total Subscription Attrition,EB10 - Creative,Canada (MA),EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,MA


In [13]:
df_DME['market_area'] = df_DME['market_area'].apply(lambda st: st[0:st.find("(")-1])
df_DME.head(5)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens
0,Total Subscription Attrition,EB10 - Creative,AMER,COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G
1,Total Subscription Attrition,EB10 - Creative,AMER,EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G
2,Total Subscription Attrition,EB10 - Creative,AMER,GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G
3,Total Subscription Attrition,EB10 - Creative,AMER,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G
4,Total Subscription Attrition,EB10 - Creative,North America,COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R


In [14]:
df_DME['market_area'].value_counts()

Korea                   264
North America           224
AMER                    224
United States           224
India                   216
ASIA                    180
ANZ                     172
Aus and New Zealand     172
SEA                     154
Southeast Asia          154
Greater China           130
EMEA                    118
China                   116
Canada                  110
Northern Europe         110
Germany                 109
Central Europe          109
Benelux                 108
Southwest Europe        108
France                  108
United Kingdom          106
Switzerland             105
Middle East             104
Italy                   104
Iberica                 104
Nordic                  102
Japan                   100
Hong Kong & Taiwan       98
SSA & Israel             88
Latin America            82
Brazil                   82
Strat. Latin America     52
JPN                      50
Eastern Europe           48
Russia & CIS             44
Mexico              

In [15]:
df_DME.head(10)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens
0,Total Subscription Attrition,EB10 - Creative,AMER,COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G
1,Total Subscription Attrition,EB10 - Creative,AMER,EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G
2,Total Subscription Attrition,EB10 - Creative,AMER,GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G
3,Total Subscription Attrition,EB10 - Creative,AMER,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G
4,Total Subscription Attrition,EB10 - Creative,North America,COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R
5,Total Subscription Attrition,EB10 - Creative,North America,EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06,R
6,Total Subscription Attrition,EB10 - Creative,North America,GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06,R
7,Total Subscription Attrition,EB10 - Creative,North America,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,R
8,Total Subscription Attrition,EB10 - Creative,Canada,COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,MA
9,Total Subscription Attrition,EB10 - Creative,Canada,EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,MA


In [16]:
df_DME['pc_ID'] = df_DME['profit_center'].apply(lambda st: st[0:st.find('-')])
df_DME['pc_descr'] = df_DME['profit_center'].apply(lambda st: st[st.find('-')+1:])

In [17]:
df_DME.sample(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr
4216,ASV & Usage Based,10850 - DCE,Southwest Europe,GOVERNMENT,Corporate,3.253999e+04,5.298771e+04,5.159058e+04,7.688241e+04,2.140007e+05,R,10850,DCE
500,Total Subscription Attrition,IS18 - Stock Photography,Iberica,COMMERCIAL,Corporate,-2.022682e+04,0.000000e+00,-8.964733e+03,-2.482847e+03,-3.167440e+04,MA,IS18,Stock Photography
2411,Net ACV,IS15 - Acrobat DC,Southeast Asia,COMMERCIAL,Corporate,1.300000e+05,1.625000e+05,1.787500e+05,1.950000e+05,6.662500e+05,MA,IS15,Acrobat DC
2764,Net ACV,IS17 - Adobe Sign,Northern Europe,EDUCATION,Corporate,-1.704069e+03,0.000000e+00,0.000000e+00,-6.624802e+02,-2.366549e+03,R,IS17,Adobe Sign
342,Total Subscription Attrition,10100 - Design,Strat. Latin America,EDUCATION,Corporate,0.000000e+00,-2.786830e+03,-1.417209e+04,-4.343230e+03,-2.130215e+04,MA,10100,Design
3644,ASV & Usage Based,13450 - Stock Photography,Italy,EDUCATION,Corporate,0.000000e+00,0.000000e+00,0.000000e+00,7.781496e+02,7.781496e+02,MA,13450,Stock Photography
694,Total Subscription Attrition,GP15 - Document Cloud,Strat. Latin America,COMMERCIAL,Corporate,-2.979773e+03,-1.201923e+03,0.000000e+00,0.000000e+00,-4.181696e+03,MA,GP15,Document Cloud
2147,Net ACV,EB15 - Document Cloud,Benelux,EDUCATION,Corporate,0.000000e+00,4.934869e+03,0.000000e+00,8.629193e+03,1.356406e+04,MA,EB15,Document Cloud
3179,ASV & Usage Based,IS10 - Creative - Professional,United States,EDUCATION,Strategic,3.885035e+06,3.796226e+06,5.797010e+06,6.521729e+06,2.000000e+07,MA,IS10,Creative - Professional
2148,Net ACV,EB15 - Document Cloud,Benelux,GOVERNMENT,Corporate,1.245769e+05,1.600378e+05,2.340007e+05,2.414112e+05,7.600266e+05,MA,EB15,Document Cloud


adding columns geo, region, market_area (which already exists)

In [18]:
df_DME['geo'] = df_DME[df_DME['in_parens']=='G']['market_area']

In [19]:
df_DME.tail(60)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo
4343,ASV & Usage Based,14400 - Adobe Sign,ANZ,EDUCATION,Strategic,4.091336e+04,5.561027e+04,4.691336e+04,5.557941e+04,1.990164e+05,R,14400,Adobe Sign,NaN
4344,ASV & Usage Based,14400 - Adobe Sign,ANZ,EDUCATION,Corporate,2.727558e+04,3.707351e+04,3.127558e+04,3.705294e+04,1.326776e+05,R,14400,Adobe Sign,NaN
4345,ASV & Usage Based,14400 - Adobe Sign,ANZ,GOVERNMENT,Strategic,6.018894e+04,6.518894e+04,8.467378e+04,8.585657e+04,2.959082e+05,R,14400,Adobe Sign,NaN
4346,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,COMMERCIAL,Strategic,1.379208e+05,1.407229e+05,2.215661e+05,2.098394e+05,7.100491e+05,MA,14400,Adobe Sign,NaN
4347,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,COMMERCIAL,Corporate,2.561386e+05,2.613425e+05,4.114798e+05,3.897018e+05,1.318663e+06,MA,14400,Adobe Sign,NaN
4348,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,EDUCATION,Strategic,4.091336e+04,5.561027e+04,4.691336e+04,5.557941e+04,1.990164e+05,MA,14400,Adobe Sign,NaN
4349,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,EDUCATION,Corporate,2.727558e+04,3.707351e+04,3.127558e+04,3.705294e+04,1.326776e+05,MA,14400,Adobe Sign,NaN
4350,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,GOVERNMENT,Strategic,6.018894e+04,6.518894e+04,8.467378e+04,8.585657e+04,2.959082e+05,MA,14400,Adobe Sign,NaN
4351,ASV & Usage Based,14400 - Adobe Sign,Greater China,COMMERCIAL,Strategic,3.136724e+04,2.757085e+04,4.308154e+04,2.925420e+04,1.312738e+05,R,14400,Adobe Sign,NaN
4352,ASV & Usage Based,14400 - Adobe Sign,Greater China,COMMERCIAL,Corporate,3.136724e+04,2.757085e+04,4.308154e+04,2.925420e+04,1.312738e+05,R,14400,Adobe Sign,NaN


In [20]:
df_DME['geo'] =df_DME['geo'].ffill()

In [21]:
df_DME['geo'].value_counts(dropna=False)

ASIA    1656
EMEA    1575
AMER    1022
JPN      150
Name: geo, dtype: int64

In [22]:
#df_DME.head(60)

### Comfortable that the forward fill of 'geo' worked well

### Working on the region now
Note: There will be a problem with the geo values here. They will all contain a region even if they are subtotals. This is OK because we will only be using the market area data and can recreate the region and geo data thanks to the new columns we have created


In [23]:
df_DME['region'] = df_DME[df_DME['in_parens']=='R']['market_area']
df_DME.head(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo,region
0,Total Subscription Attrition,EB10 - Creative,AMER,COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G,EB10,Creative,AMER,NaN
1,Total Subscription Attrition,EB10 - Creative,AMER,EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G,EB10,Creative,AMER,NaN
2,Total Subscription Attrition,EB10 - Creative,AMER,GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G,EB10,Creative,AMER,NaN
3,Total Subscription Attrition,EB10 - Creative,AMER,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G,EB10,Creative,AMER,NaN
4,Total Subscription Attrition,EB10 - Creative,North America,COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R,EB10,Creative,AMER,North America
5,Total Subscription Attrition,EB10 - Creative,North America,EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06,R,EB10,Creative,AMER,North America
6,Total Subscription Attrition,EB10 - Creative,North America,GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06,R,EB10,Creative,AMER,North America
7,Total Subscription Attrition,EB10 - Creative,North America,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,R,EB10,Creative,AMER,North America
8,Total Subscription Attrition,EB10 - Creative,Canada,COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,MA,EB10,Creative,AMER,NaN
9,Total Subscription Attrition,EB10 - Creative,Canada,EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,MA,EB10,Creative,AMER,NaN


In [24]:
df_DME['region'] = df_DME['region'].ffill()
df_DME['region'].value_counts(dropna=False)

Southwest Europe    598
North America       566
Northern Europe     510
SEA                 424
Latin America       418
Central Europe      415
Greater China       344
ANZ                 344
Japan               300
Korea               264
India               216
NaN                   4
Name: region, dtype: int64

In [25]:
df_DME.tail(40)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo,region
4363,ASV & Usage Based,14400 - Adobe Sign,SEA,COMMERCIAL,Strategic,1.862253e+05,2.048968e+05,2.311119e+05,2.341869e+05,8.564209e+05,R,14400,Adobe Sign,ASIA,SEA
4364,ASV & Usage Based,14400 - Adobe Sign,SEA,COMMERCIAL,Corporate,7.981083e+04,8.781293e+04,9.904795e+04,1.003658e+05,3.670375e+05,R,14400,Adobe Sign,ASIA,SEA
4365,ASV & Usage Based,14400 - Adobe Sign,SEA,EDUCATION,Strategic,1.301613e+04,1.301613e+04,1.301613e+04,2.351613e+04,6.256452e+04,R,14400,Adobe Sign,ASIA,SEA
4366,ASV & Usage Based,14400 - Adobe Sign,SEA,EDUCATION,Corporate,5.578341e+03,5.578341e+03,5.578341e+03,1.007834e+04,2.681336e+04,R,14400,Adobe Sign,ASIA,SEA
4367,ASV & Usage Based,14400 - Adobe Sign,SEA,GOVERNMENT,Strategic,0.000000e+00,0.000000e+00,1.718894e+05,2.578341e+05,4.297235e+05,R,14400,Adobe Sign,ASIA,SEA
4368,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,COMMERCIAL,Strategic,1.862253e+05,2.048968e+05,2.311119e+05,2.341869e+05,8.564209e+05,MA,14400,Adobe Sign,ASIA,SEA
4369,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,COMMERCIAL,Corporate,7.981083e+04,8.781293e+04,9.904795e+04,1.003658e+05,3.670375e+05,MA,14400,Adobe Sign,ASIA,SEA
4370,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,EDUCATION,Strategic,1.301613e+04,1.301613e+04,1.301613e+04,2.351613e+04,6.256452e+04,MA,14400,Adobe Sign,ASIA,SEA
4371,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,EDUCATION,Corporate,5.578341e+03,5.578341e+03,5.578341e+03,1.007834e+04,2.681336e+04,MA,14400,Adobe Sign,ASIA,SEA
4372,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,GOVERNMENT,Strategic,0.000000e+00,0.000000e+00,1.718894e+05,2.578341e+05,4.297235e+05,MA,14400,Adobe Sign,ASIA,SEA


#### Now we only want to keep the market areas

In [26]:
df_DME = df_DME[df_DME['in_parens']=='MA'].copy()
df_DME['market_area'].value_counts(dropna=False)

United States           224
Aus and New Zealand     172
Southeast Asia          154
Korea                   132
China                   116
Canada                  110
Germany                 109
Benelux                 108
India                   108
France                  108
United Kingdom          106
Switzerland             105
Iberica                 104
Italy                   104
Middle East             104
Nordic                  102
Hong Kong & Taiwan       98
SSA & Israel             88
Brazil                   82
Strat. Latin America     52
Japan                    50
Eastern Europe           48
Russia & CIS             44
Mexico                   24
Name: market_area, dtype: int64

In [27]:
len(df_DME)

2452

In [28]:
df_DME.sample(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo,region
1391,Net ACV,GP10 - Creative,SSA & Israel,COMMERCIAL,Corporate,19015.956683,-89986.282156,20487.435520,6.783017e+04,1.734728e+04,MA,GP10,Creative,EMEA,Northern Europe
245,Total Subscription Attrition,IS10 - Creative - Professional,Aus and New Zealand,EDUCATION,Strategic,-9717.701377,-47011.144870,-1813.642302,-5.784131e+04,-1.163838e+05,MA,IS10,Creative - Professional,ASIA,ANZ
708,Total Subscription Attrition,GP15 - Document Cloud,Hong Kong & Taiwan,COMMERCIAL,Strategic,-11921.144012,-27118.771988,-12245.460000,-3.286461e+03,-5.457184e+04,MA,GP15,Document Cloud,ASIA,Greater China
417,Total Subscription Attrition,10100 - Design,Iberica,EDUCATION,Corporate,0.000000,0.000000,0.000000,-3.595674e+04,-3.595674e+04,MA,10100,Design,EMEA,Southwest Europe
1810,Net ACV,10110 - CCE + Stock,Italy,COMMERCIAL,Corporate,66076.332000,61129.142860,139212.979059,7.050687e+04,3.369253e+05,MA,10110,CCE + Stock,EMEA,Southwest Europe
446,Total Subscription Attrition,IS18 - Stock Photography,United States,NON-PROFIT,Corporate,-75.475000,0.000000,-3690.214186,-2.806804e+03,-6.572493e+03,MA,IS18,Stock Photography,AMER,North America
1241,Net ACV,EB10 - Creative,Benelux,GOVERNMENT,Corporate,135781.541298,149982.641052,173396.604533,3.222390e+05,7.813998e+05,MA,EB10,Creative,EMEA,Southwest Europe
3123,ASV & Usage Based,GP10 - Creative,Germany,EDUCATION,Corporate,182297.338186,200928.321749,270074.206532,7.482869e+05,1.401587e+06,MA,GP10,Creative,EMEA,Central Europe
2229,Net ACV,GP15 - Document Cloud,China,COMMERCIAL,Strategic,560210.313565,551168.907492,547045.662246,6.541541e+05,2.312579e+06,MA,GP15,Document Cloud,ASIA,Greater China
2481,Net ACV,10800 - Acrobat Desk,Canada,EDUCATION,Corporate,0.000000,0.000000,0.000000,-1.401371e+03,-1.401371e+03,MA,10800,Acrobat Desk,AMER,North America


In [29]:
df_DME = df_DME.drop(columns=['profit_center', 'in_parens' ])

In [30]:
df_DME.head()

,metrics,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,pc_ID,pc_descr,geo,region
8,Total Subscription Attrition,Canada,COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,EB10,Creative,AMER,North America
9,Total Subscription Attrition,Canada,EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,EB10,Creative,AMER,North America
10,Total Subscription Attrition,Canada,GOVERNMENT,Corporate,-1.073110e+05,-5.261821e+04,-4.332915e+04,-1.879336e+04,-2.220517e+05,EB10,Creative,AMER,North America
11,Total Subscription Attrition,United States,COMMERCIAL,Corporate,-2.763076e+06,-1.584732e+06,-2.420743e+06,-2.622828e+06,-9.391380e+06,EB10,Creative,AMER,North America
12,Total Subscription Attrition,United States,EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.167310e+06,-4.610124e+05,-2.125721e+06,EB10,Creative,AMER,North America


In [31]:
df_DME.columns

Index(['metrics', 'market_area', 'segment', 'GTM', 'Q1_2021', 'Q2_2021',
       'Q3_2021', 'Q4_2021', '2021', 'pc_ID', 'pc_descr', 'geo', 'region'],
      dtype='object')

In [32]:
df_DME['metrics'].value_counts()

Net ACV                         990
ASV & Usage Based               822
Total Subscription Attrition    640
Name: metrics, dtype: int64

##### We only want the 'Net ACV' bookings here, so we need to remove any other bookings type

In [33]:
df_DME = df_DME[df_DME['metrics']=='Net ACV']

In [34]:
df_DME.sample(10)

,metrics,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,pc_ID,pc_descr,geo,region
2703,Net ACV,United States,COMMERCIAL,Strategic,4.088012e+06,5.664615e+06,8.755720e+06,1.107230e+07,2.958065e+07,IS17,Adobe Sign,AMER,North America
1117,Net ACV,United States,EDUCATION,Strategic,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,2.027118e+07,EB10,Creative,AMER,North America
2758,Net ACV,Germany,GOVERNMENT,Corporate,4.156231e+03,5.000000e+04,2.811544e+04,5.250152e+04,1.347732e+05,IS17,Adobe Sign,EMEA,Central Europe
2025,Net ACV,Canada,COMMERCIAL,Strategic,6.226860e+05,6.782545e+05,9.898028e+05,1.499058e+06,3.789801e+06,EB15,Document Cloud,AMER,North America
2502,Net ACV,Strat. Latin America,GOVERNMENT,Corporate,-1.813826e+03,0.000000e+00,0.000000e+00,0.000000e+00,-1.813826e+03,10800,Acrobat Desk,AMER,Latin America
1696,Net ACV,Germany,GOVERNMENT,Corporate,3.651596e+05,5.372816e+05,9.444298e+05,1.058117e+06,2.904988e+06,10100,Design,EMEA,Central Europe
2580,Net ACV,Nordic,GOVERNMENT,Corporate,2.610646e+04,6.772817e+04,6.797186e+04,1.684025e+05,3.302090e+05,10800,Acrobat Desk,EMEA,Northern Europe
2773,Net ACV,Middle East,COMMERCIAL,Corporate,-4.333676e+01,2.303640e+04,4.773120e+04,1.315274e+05,2.022517e+05,IS17,Adobe Sign,EMEA,Northern Europe
1481,Net ACV,China,COMMERCIAL,Strategic,2.892967e+06,3.395373e+06,3.473089e+06,3.627866e+06,1.338930e+07,IS10,Creative - Professional,ASIA,Greater China
1510,Net ACV,Korea,COMMERCIAL,Corporate,9.000000e+05,9.400000e+05,9.400000e+05,9.560000e+05,3.736000e+06,IS10,Creative - Professional,ASIA,Korea


In [35]:
df_DME = df_DME[['pc_descr', 'pc_ID', 'geo', 'region', 'market_area', 'segment', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021', '2021']]

In [36]:
df_DME.head(10)

,pc_descr,pc_ID,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
1110,Creative,EB10,AMER,North America,Canada,COMMERCIAL,6.095483e+05,6.234444e+05,9.189459e+05,9.843834e+05,3.136322e+06
1111,Creative,EB10,AMER,North America,Canada,COMMERCIAL,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05
1112,Creative,EB10,AMER,North America,Canada,EDUCATION,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05
1113,Creative,EB10,AMER,North America,Canada,GOVERNMENT,3.357112e+05,3.509312e+05,5.147951e+05,5.522203e+05,1.753658e+06
1114,Creative,EB10,AMER,North America,Canada,GOVERNMENT,-1.073110e+05,-5.261821e+04,-4.332915e+04,-1.879336e+04,-2.220517e+05
1115,Creative,EB10,AMER,North America,United States,COMMERCIAL,1.102294e+07,1.144855e+07,1.671312e+07,1.767977e+07,5.686438e+07
1116,Creative,EB10,AMER,North America,United States,COMMERCIAL,-3.276029e+05,9.436072e+05,1.282755e+06,1.322818e+06,3.221577e+06
1117,Creative,EB10,AMER,North America,United States,EDUCATION,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,2.027118e+07
1118,Creative,EB10,AMER,North America,United States,EDUCATION,-2.251662e+05,-2.722325e+05,-1.167310e+06,-4.610124e+05,-2.125721e+06
1119,Creative,EB10,AMER,North America,United States,GOVERNMENT,1.944009e+06,1.933252e+06,2.929489e+06,3.274859e+06,1.008161e+07


In [37]:
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 1110 to 2892
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pc_descr     990 non-null    object 
 1   pc_ID        990 non-null    object 
 2   geo          990 non-null    object 
 3   region       990 non-null    object 
 4   market_area  990 non-null    object 
 5   segment      990 non-null    object 
 6   Q1_2021      990 non-null    float64
 7   Q2_2021      990 non-null    float64
 8   Q3_2021      990 non-null    float64
 9   Q4_2021      990 non-null    float64
 10  2021         990 non-null    float64
dtypes: float64(5), object(6)
memory usage: 92.8+ KB


### The DX bookings have no information about segment. We need to delete segment and then group by to following
- pc_descr
- geo
- region
- market_area


In [38]:
df_DME = df_DME.drop(columns = 'segment')
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 1110 to 2892
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pc_descr     990 non-null    object 
 1   pc_ID        990 non-null    object 
 2   geo          990 non-null    object 
 3   region       990 non-null    object 
 4   market_area  990 non-null    object 
 5   Q1_2021      990 non-null    float64
 6   Q2_2021      990 non-null    float64
 7   Q3_2021      990 non-null    float64
 8   Q4_2021      990 non-null    float64
 9   2021         990 non-null    float64
dtypes: float64(5), object(5)
memory usage: 85.1+ KB


In [39]:
df_DME.sum()

pc_descr        Creative Creative Creative Creative Creative ...
pc_ID          EB10 EB10 EB10 EB10 EB10 EB10 EB10 EB10 EB10 E...
geo            AMERAMERAMERAMERAMERAMERAMERAMERAMERAMERAMERAM...
region         North AmericaNorth AmericaNorth AmericaNorth A...
market_area    CanadaCanadaCanadaCanadaCanadaUnited StatesUni...
Q1_2021                                              3.35128e+08
Q2_2021                                              4.02485e+08
Q3_2021                                              4.98973e+08
Q4_2021                                               6.7508e+08
2021                                                 1.91167e+09
dtype: object

In [40]:
df_DME = df_DME.groupby(by = ['pc_descr', 'geo', 'region', 'market_area']).sum()
df_DME.sum()

Q1_2021    3.351281e+08
Q2_2021    4.024845e+08
Q3_2021    4.989733e+08
Q4_2021    6.750802e+08
2021       1.911666e+09
dtype: float64

In [41]:
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 227 entries, (' Acrobat DC', 'AMER', 'Latin America', 'Brazil') to (' Stock Photography', 'JPN', 'Japan', 'Japan')
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Q1_2021  227 non-null    float64
 1   Q2_2021  227 non-null    float64
 2   Q3_2021  227 non-null    float64
 3   Q4_2021  227 non-null    float64
 4   2021     227 non-null    float64
dtypes: float64(5)
memory usage: 10.4+ KB


In [42]:
df_DME.head(10)

Q1_2021  \
pc_descr    geo  region        market_area                          
 Acrobat DC AMER Latin America Brazil                1.617778e+05   
                               Mexico               -5.878793e+02   
                               Strat. Latin America -4.793599e+03   
                 North America Canada                7.286022e+05   
                               United States         9.789297e+06   
            ASIA ANZ           Aus and New Zealand   1.006561e+06   
                 Greater China China                 1.085210e+06   
                               Hong Kong & Taiwan    1.573385e+05   
                 India         India                 4.941753e+05   
                 Korea         Korea                 1.649261e+05   

                                                          Q2_2021  \
pc_descr    geo  region        market_area                          
 Acrobat DC AMER Latin America Brazil                1.777746e+05   
                               Mexico                0.000000e+00   
                               Strat. Latin America -1.201923e+03   
                 North America Canada                8.484676e+05   
                               United States         1.080222e+07   
            ASIA ANZ           Aus and New Zealand   1.162570e+06   
                 Greater China China                 1.131169e+06   
                               Hong Kong & Taiwan    1.648655e+05   
                 India         India                 4.768403e+05   
                 Korea         Korea                 2.844877e+05   

                                                          Q3_2021  \
pc_descr    geo  region        market_area                          
 Acrobat DC AMER Latin America Brazil                2.596567e+05   
                               Mexico                0.000000e+00   
                               Strat. Latin America -2.246623e+01   
                 North America Canada                1.233965e+06   
                               United States         1.504463e+07   
            ASIA ANZ           Aus and New Zealand   1.295138e+06   
                 Greater China China                 1.128201e+06   
                               Hong Kong & Taiwan    1.902880e+05   
                 India         India                 6.236849e+05   
                 Korea         Korea                 3.649871e+05   

                                                          Q4_2021  \
pc_descr    geo  region        market_area                          
 Acrobat DC AMER Latin America Brazil                3.974719e+05   
                               Mexico                0.000000e+00   
                               Strat. Latin America  0.000000e+00   
                 North America Canada                1.953663e+06   
                               United States         2.315904e+07   
            ASIA ANZ           Aus and New Zealand   1.698991e+06   
                 Greater China China                 1.329854e+06   
                               Hong Kong & Taiwan    1.894954e+05   
                 India         India                 6.620033e+05   
                 Korea         Korea                 3.934598e+05   

                                                             2021  
pc_descr    geo  region        market_area                         
 Acrobat DC AMER Latin America Brazil                9.966810e+05  
                               Mexico               -5.878793e+02  
                               Strat. Latin America -6.017988e+03  
                 North America Canada                4.764697e+06  
                               United States         5.879518e+07  
            ASIA ANZ           Aus and New Zealand   5.163260e+06  
                 Greater China China                 4.674434e+06  
                               Hong Kong & Taiwan    7.019874e+05  
                 India         India                 2.256704e

In [43]:
df_DME.reset_index(inplace=True)


### We need to add a "BU" and "segment" field to these
The BU will be Digital Media and the segment will all be lumped into Creative


In [44]:
df_DME['BU'] = 'Digital Media'
df_DME['segment'] = 'Creative'

In [45]:
df_DME.head(10)

,pc_descr,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,BU,segment
0,Acrobat DC,AMER,Latin America,Brazil,1.617778e+05,1.777746e+05,2.596567e+05,3.974719e+05,9.966810e+05,Digital Media,Creative
1,Acrobat DC,AMER,Latin America,Mexico,-5.878793e+02,0.000000e+00,0.000000e+00,0.000000e+00,-5.878793e+02,Digital Media,Creative
2,Acrobat DC,AMER,Latin America,Strat. Latin America,-4.793599e+03,-1.201923e+03,-2.246623e+01,0.000000e+00,-6.017988e+03,Digital Media,Creative
3,Acrobat DC,AMER,North America,Canada,7.286022e+05,8.484676e+05,1.233965e+06,1.953663e+06,4.764697e+06,Digital Media,Creative
4,Acrobat DC,AMER,North America,United States,9.789297e+06,1.080222e+07,1.504463e+07,2.315904e+07,5.879518e+07,Digital Media,Creative
5,Acrobat DC,ASIA,ANZ,Aus and New Zealand,1.006561e+06,1.162570e+06,1.295138e+06,1.698991e+06,5.163260e+06,Digital Media,Creative
6,Acrobat DC,ASIA,Greater China,China,1.085210e+06,1.131169e+06,1.128201e+06,1.329854e+06,4.674434e+06,Digital Media,Creative
7,Acrobat DC,ASIA,Greater China,Hong Kong & Taiwan,1.573385e+05,1.648655e+05,1.902880e+05,1.894954e+05,7.019874e+05,Digital Media,Creative
8,Acrobat DC,ASIA,India,India,4.941753e+05,4.768403e+05,6.236849e+05,6.620033e+05,2.256704e+06,Digital Media,Creative
9,Acrobat DC,ASIA,Korea,Korea,1.649261e+05,2.844877e+05,3.649871e+05,3.934598e+05,1.207861e+06,Digital Media,Creative


In [46]:
df_DME = df_DME[['BU', 'segment', 'pc_descr','geo', 'region', 'market_area', 'segment', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021', '2021']]

In [47]:
df_DME.pc_descr.value_counts(dropna=False)

 Acrobat DC                 24
 Document Cloud             24
 Acrobat Desk               24
 Creative - Professional    24
 Creative                   24
 Design                     24
 Stock Photography          22
 Adobe Sign                 21
 CCE + Stock                21
 DCE                        19
Name: pc_descr, dtype: int64

## Done with DME now moving to DX



In [48]:
filename = r'/Volumes/Treasury/Financial_Database/Deferred_Revenue/Inputs/DATA_2020_p12/DX_Bookings_FY21_Plan.xlsx'
sheetname = 'Sheet1'
start=12
df_DX = pd.read_excel(filename, sheetname, skiprows=start)

In [49]:
df_DX.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021,Q1 2021,Q2 2021,Q3 2021,Q4 2021
0,Enterprise,All Market Areas,12400 - Adobe Analytics,7.223474e+07,1.316699e+07,1.652153e+07,1.814789e+07,2.439833e+07
1,Enterprise,All Market Areas,14800 - Customer Journey Analytics,2.861741e+07,5.059796e+06,6.665272e+06,6.952678e+06,9.939666e+06
2,Enterprise,All Market Areas,12610 - Audience Manager,1.028093e+07,1.761821e+06,2.332947e+06,2.492824e+06,3.693333e+06
3,Enterprise,All Market Areas,14700 - Real Time CDP,3.093418e+07,5.656254e+06,6.457362e+06,7.532753e+06,1.128781e+07
4,Enterprise,All Market Areas,12500 - Adobe Target,4.010254e+07,7.320283e+06,9.058720e+06,9.848370e+06,1.387517e+07
5,Enterprise,All Market Areas,14600 - Adobe Exp Platform,5.381726e+07,9.536363e+06,1.243553e+07,1.317641e+07,1.866895e+07
6,Enterprise,All Market Areas,13010 - Assets,7.243556e+07,1.310389e+07,1.560123e+07,1.810836e+07,2.562207e+07
7,Enterprise,All Market Areas,13020 - Sites,1.803132e+08,3.175266e+07,4.029789e+07,4.440377e+07,6.385891e+07
8,Enterprise,All Market Areas,13030 - Forms,3.738079e+07,6.195738e+06,8.025811e+06,9.466740e+06,1.369250e+07
9,Enterprise,All Market Areas,12630 - Magento,6.785980e+07,1.224868e+07,1.467157e+07,1.674743e+07,2.419212e+07


In [50]:
df_DX = df_DX.rename(columns = {'Unnamed: 0': 'segment',
                                'Unnamed: 1': 'market_area',
                                'Unnamed: 2': 'profit_center',
                                'Q1 2021':'Q1_2021',
                                'Q2 2021':'Q2_2021',
                                'Q3 2021':'Q3_2021',
                                'Q4 2021':'Q4_2021' 
                                })

## It looks like there are three seperate cuts of the data that are a hierarchy again, but they also have an 'All Market Areas' that is a total by profit center. We don't need this but there is no need to change the code here.
It will be removed when we get to the filter on region

1. G is GEO
2. R is region
3. MA is market Area

Going to split these into seperate sections and do successive fill forwards to cover all of them



In [51]:
df_DX.head()

,segment,market_area,profit_center,2021,Q1_2021,Q2_2021,Q3_2021,Q4_2021
0,Enterprise,All Market Areas,12400 - Adobe Analytics,7.223474e+07,1.316699e+07,1.652153e+07,1.814789e+07,2.439833e+07
1,Enterprise,All Market Areas,14800 - Customer Journey Analytics,2.861741e+07,5.059796e+06,6.665272e+06,6.952678e+06,9.939666e+06
2,Enterprise,All Market Areas,12610 - Audience Manager,1.028093e+07,1.761821e+06,2.332947e+06,2.492824e+06,3.693333e+06
3,Enterprise,All Market Areas,14700 - Real Time CDP,3.093418e+07,5.656254e+06,6.457362e+06,7.532753e+06,1.128781e+07
4,Enterprise,All Market Areas,12500 - Adobe Target,4.010254e+07,7.320283e+06,9.058720e+06,9.848370e+06,1.387517e+07


In [52]:
# identify the characters in a string
df_DX['in_parens'] =  df_DX['market_area'].apply(lambda st: st[st.find("(")+1:st.find(")")])

In [53]:
df_DX['market_area'] = df_DX['market_area'].apply(lambda st: st[0:st.find("(")-1])

In [54]:
df_DX['pc_ID'] = df_DX['profit_center'].apply(lambda st: st[0:st.find('-')])
df_DX['pc_descr'] = df_DX['profit_center'].apply(lambda st: st[st.find('-')+1:])

In [55]:
df_DX['geo'] = df_DX[df_DX['in_parens']=='G']['market_area']
df_DX['geo'] =df_DX['geo'].ffill()

In [56]:
df_DX['region'] = df_DX[df_DX['in_parens']=='R']['market_area']
df_DX['region'] = df_DX['region'].ffill()

In [57]:
# filter to just include market area
df_DX = df_DX[df_DX['in_parens']=='MA'].copy()

In [58]:
# drop unnecessary columns and reorder the columns
df_DX = df_DX.drop(columns=['profit_center', 'in_parens' ])
df_DX = df_DX[['pc_descr', 'pc_ID', 'geo', 'region', 'market_area', 'segment', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021', '2021']]

In [59]:
# We need to remove the segment data: it is not included in the DME bookings
df_DX = df_DX.groupby(by = ['pc_descr', 'geo', 'region', 'market_area']).sum()
df_DX = df_DX.reset_index()


In [60]:
# need to add the BU and segment information to the bookings file so that it matches
df_DX['BU'] = 'Digital Experience'
df_DX['segment'] = 'Experience Cloud'



In [61]:
df_DX = df_DX[['BU', 'segment', 'pc_descr','geo', 'region', 'market_area', 'segment', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021', '2021']]

In [62]:
df_DX.head(20)

,BU,segment,pc_descr,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
0,Digital Experience,Experience Cloud,Adobe Analytics,AMER,Latin America,Brazil,Experience Cloud,5.513752e+05,3.867542e+05,8.171629e+05,1.194088e+06,2.949380e+06
1,Digital Experience,Experience Cloud,Adobe Analytics,AMER,North America,Canada,Experience Cloud,1.188265e+06,1.672009e+06,1.602847e+06,1.959167e+06,6.422288e+06
2,Digital Experience,Experience Cloud,Adobe Analytics,AMER,North America,United States,Experience Cloud,2.015296e+07,2.452051e+07,2.725794e+07,3.492371e+07,1.068551e+08
3,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,ANZ,Aus and New Zealand,Experience Cloud,4.536448e+05,6.480639e+05,8.100799e+05,1.328531e+06,3.240320e+06
4,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,Greater China,China,Experience Cloud,1.351974e+05,2.027961e+05,2.196958e+05,2.872945e+05,8.449839e+05
5,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,Greater China,Hong Kong & Taiwan,Experience Cloud,1.332610e+05,1.998916e+05,2.165492e+05,2.831797e+05,8.328815e+05
6,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,India,India,Experience Cloud,8.381679e+05,1.024427e+06,1.303817e+06,1.490076e+06,4.656488e+06
7,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,Korea,Korea,Experience Cloud,1.353208e+05,2.029812e+05,2.198963e+05,2.875566e+05,8.457548e+05
8,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,SEA,Southeast Asia,Experience Cloud,3.941892e+05,5.101272e+05,6.028776e+05,8.115660e+05,2.318760e+06
9,Digital Experience,Experience Cloud,Adobe Analytics,EMEA,Central Europe,Eastern Europe,Experience Cloud,7.295550e+03,9.575409e+03,1.048735e+04,1.823887e+04,4.559718e+04


In [63]:
df_DX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BU           277 non-null    object 
 1   segment      277 non-null    object 
 2   pc_descr     277 non-null    object 
 3   geo          277 non-null    object 
 4   region       277 non-null    object 
 5   market_area  277 non-null    object 
 6   segment      277 non-null    object 
 7   Q1_2021      277 non-null    float64
 8   Q2_2021      277 non-null    float64
 9   Q3_2021      277 non-null    float64
 10  Q4_2021      277 non-null    float64
 11  2021         277 non-null    float64
dtypes: float64(5), object(7)
memory usage: 26.1+ KB


# Final Check on the data


In [64]:
# DX
df_DX.sample(40)

,BU,segment,pc_descr,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
43,Digital Experience,Experience Cloud,Adobe Campaign,EMEA,Southwest Europe,Italy,Experience Cloud,3.061800e+05,4.252500e+05,3.231900e+05,6.463800e+05,1.701000e+06
256,Digital Experience,Experience Cloud,Sites,AMER,North America,United States,Experience Cloud,3.942681e+07,4.525577e+07,5.284945e+07,6.860959e+07,2.061416e+08
275,Digital Experience,Experience Cloud,Sites,EMEA,Southwest Europe,Italy,Experience Cloud,3.996000e+05,5.550000e+05,4.218000e+05,8.436000e+05,2.220000e+06
240,Digital Experience,Experience Cloud,Real Time CDP,AMER,North America,United States,Experience Cloud,7.595319e+06,7.491295e+06,9.727710e+06,1.341097e+07,3.822530e+07
118,Digital Experience,Experience Cloud,Audience Manager,EMEA,Northern Europe,Nordic,Experience Cloud,1.023757e+05,1.638012e+05,1.535636e+05,2.627644e+05,6.825050e+05
8,Digital Experience,Experience Cloud,Adobe Analytics,ASIA,SEA,Southeast Asia,Experience Cloud,3.941892e+05,5.101272e+05,6.028776e+05,8.115660e+05,2.318760e+06
198,Digital Experience,Experience Cloud,Marketo,ASIA,India,India,Experience Cloud,1.080000e+05,1.320000e+05,1.680000e+05,1.920000e+05,6.000000e+05
170,Digital Experience,Experience Cloud,Journeys,EMEA,Southwest Europe,Iberica,Experience Cloud,4.680000e+04,6.500000e+04,4.940000e+04,9.880000e+04,2.600000e+05
68,Digital Experience,Experience Cloud,Adobe Target,ASIA,Korea,Korea,Experience Cloud,1.613786e+05,2.420679e+05,2.622402e+05,3.429295e+05,1.008616e+06
206,Digital Experience,Experience Cloud,Marketo,EMEA,Northern Europe,SSA & Israel,Experience Cloud,1.201248e+04,1.921996e+04,1.801872e+04,3.083202e+04,8.008318e+04


In [65]:
# DME
df_DME.sample(40)

,BU,segment,pc_descr,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
29,Digital Media,Creative,Acrobat Desk,ASIA,ANZ,Aus and New Zealand,Creative,8.473553e+05,9.280298e+05,1.027319e+06,1.221180e+06,4.023884e+06
112,Digital Media,Creative,Creative,EMEA,Southwest Europe,Italy,Creative,2.765179e+05,2.537284e+05,7.323590e+05,7.829332e+05,2.045539e+06
146,Digital Media,Creative,DCE,EMEA,Central Europe,Eastern Europe,Creative,2.442372e+03,1.107600e+03,0.000000e+00,1.252515e+04,1.607512e+04
206,Digital Media,Creative,Stock Photography,AMER,Latin America,Strat. Latin America,Creative,0.000000e+00,0.000000e+00,0.000000e+00,-1.181812e+04,-1.181812e+04
81,Digital Media,Creative,CCE + Stock,EMEA,Northern Europe,Middle East,Creative,1.349502e+04,0.000000e+00,0.000000e+00,5.430514e+04,6.780016e+04
9,Digital Media,Creative,Acrobat DC,ASIA,Korea,Korea,Creative,1.649261e+05,2.844877e+05,3.649871e+05,3.934598e+05,1.207861e+06
139,Digital Media,Creative,DCE,AMER,North America,Canada,Creative,2.473430e+05,2.548851e+05,4.095525e+05,6.312123e+05,1.542993e+06
141,Digital Media,Creative,DCE,ASIA,ANZ,Aus and New Zealand,Creative,1.592057e+05,2.345407e+05,2.678188e+05,4.778106e+05,1.139376e+06
98,Digital Media,Creative,Creative,ASIA,India,India,Creative,4.325554e+06,4.447427e+06,4.905994e+06,6.214906e+06,1.989388e+07
169,Digital Media,Creative,Design,EMEA,Central Europe,Germany,Creative,1.056981e+06,1.561883e+06,2.967159e+06,4.487416e+06,1.007344e+07


In [66]:
df_DME.sample(5)

,BU,segment,pc_descr,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
109,Digital Media,Creative,Creative,EMEA,Southwest Europe,Benelux,Creative,2.867065e+06,1.912516e+06,2.179015e+06,4.660655e+06,1.161925e+07
59,Digital Media,Creative,Adobe Sign,EMEA,Central Europe,Switzerland,Creative,1.344376e+05,4.096195e+05,3.850066e+05,6.588908e+05,1.587955e+06
63,Digital Media,Creative,Adobe Sign,EMEA,Northern Europe,United Kingdom,Creative,3.105466e+06,4.255451e+06,4.818470e+06,9.423434e+06,2.160282e+07
163,Digital Media,Creative,Design,ASIA,Greater China,China,Creative,6.390890e+06,7.218400e+06,7.461772e+06,7.697756e+06,2.876882e+07
221,Digital Media,Creative,Stock Photography,EMEA,Northern Europe,United Kingdom,Creative,-4.833583e+05,3.174812e+05,3.599139e+05,4.953324e+05,6.893693e+05


In [67]:
df_DX.sample(5)

,BU,segment,pc_descr,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
262,Digital Experience,Experience Cloud,Sites,ASIA,SEA,Southeast Asia,Experience Cloud,1.558349e+06,2.821329e+06,2.152848e+06,2.897535e+06,9.430061e+06
171,Digital Experience,Experience Cloud,Magento,AMER,Latin America,Brazil,Experience Cloud,2.916360e+05,2.045638e+05,4.322177e+05,6.315825e+05,1.560000e+06
30,Digital Experience,Experience Cloud,Adobe Campaign,ASIA,Korea,Korea,Experience Cloud,2.206079e+05,3.309118e+05,3.584878e+05,4.687917e+05,1.378799e+06
264,Digital Experience,Experience Cloud,Sites,EMEA,Central Europe,Germany,Experience Cloud,2.408824e+06,3.161582e+06,3.462685e+06,6.022060e+06,1.505515e+07
89,Digital Experience,Experience Cloud,Assets,ASIA,India,India,Experience Cloud,3.645480e+05,4.455587e+05,5.670747e+05,6.480854e+05,2.025267e+06


# Checking totals


In [68]:
df_DX.sum()

BU             Digital ExperienceDigital ExperienceDigital Ex...
segment        Experience CloudExperience CloudExperience Clo...
pc_descr        Adobe Analytics Adobe Analytics Adobe Analyti...
geo            AMERAMERAMERASIAASIAASIAASIAASIAASIAEMEAEMEAEM...
region         Latin AmericaNorth AmericaNorth AmericaANZGrea...
market_area    BrazilCanadaUnited StatesAus and New ZealandCh...
segment        Experience CloudExperience CloudExperience Clo...
Q1_2021                                              2.88639e+08
Q2_2021                                              3.58238e+08
Q3_2021                                              3.98039e+08
Q4_2021                                              5.68557e+08
2021                                                 1.61347e+09
dtype: object

In [69]:
df_DME.sum()

BU             Digital MediaDigital MediaDigital MediaDigital...
segment        CreativeCreativeCreativeCreativeCreativeCreati...
pc_descr        Acrobat DC Acrobat DC Acrobat DC Acrobat DC A...
geo            AMERAMERAMERAMERAMERASIAASIAASIAASIAASIAASIAEM...
region         Latin AmericaLatin AmericaLatin AmericaNorth A...
market_area    BrazilMexicoStrat. Latin AmericaCanadaUnited S...
segment        CreativeCreativeCreativeCreativeCreativeCreati...
Q1_2021                                              3.35128e+08
Q2_2021                                              4.02485e+08
Q3_2021                                              4.98973e+08
Q4_2021                                               6.7508e+08
2021                                                 1.91167e+09
dtype: object